In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# %cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA
%cd /gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA

/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA


Setting up path to the data

In [ ]:
# data_path = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
data_path = '/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/raw/'
# Document id and Answer text
collection = load_answers_to_df(data_path+"FiQA_train_doc_final.tsv")
# Question id and Question text
queries = load_questions_to_df(data_path+"FiQA_train_question_final.tsv")
# Question id and Answer id pair
qid_docid = load_qid_docid_to_df(data_path+"FiQA_train_question_doc_final.tsv")

Install package for answer retrieval system

In [ ]:
!pip install pyserini
!pip install faiss

!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.5 MB 38 kB/s 
     |████████████████████████████████| 1.3 MB 51.9 MB/s 
     |████████████████████████████████| 13.5 MB 57.6 MB/s 
     |████████████████████████████████| 2.0 MB 43.5 MB/s 
     |████████████████████████████████| 4.7 MB 59.1 MB/s 
     |████████████████████████████████| 4.9 MB 66.0 MB/s 
     |████████████████████████████████| 1.2 MB 70.0 MB/s 
     |████████████████████████████████| 188 kB 68.3 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
 

In [ ]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet
!pip install pydot --quiet

Import packages

In [ ]:
#from pyserini.search import SimpleSearcher

import pandas as pd
import regex as re
import csv
from itertools import islice
import pickle
import numpy as np
import json
import os
import sys
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
from pathlib import Path
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#from pyserini.search as pysearch

from pyserini.search import SimpleSearcher
from src.utils import *
from statistics import mean 
from src.process_data import *

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt

import re

In [ ]:
#Parameters for model testing
max_length=256
top_k = 2
labels_count = 3

Prepare the dataset for training, validation and test 

In [ ]:
fiqa_index = "/gdrive/MyDrive//w266-final-project-master/w266-final-project-master/FinBERT-QA/retriever/lucene-index-fiqa/"


def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def create_dataset(question_df, labels, cands_size):
    """Retrieves the top-k candidate answers for a question and
    creates a list of lists of the dataset containing the question id,
    list of relevant answer ids, and the list of answer candidates

    Returns:
        dataset: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        question_df: Dataframe containing the qid and question text
        labels: Dictonary containing the qid to text map
        cands_size: int - number of candidates to retrieve
    """
    dataset = []
    # Calls retriever
    searcher = SimpleSearcher(fiqa_index)
    # For each question
    for i, row in question_df.iterrows():
        qid = row['qid']
        tmp = []
        # Append qid
        tmp.append(qid)
        # Append list of relevant docs
        tmp.append(labels[qid])
        # Retrieves answer candidates
        cands = []
        cands_score = []
        query = row['question']
        query = re.sub('[£€§]', '', query)
        hits = searcher.search(query, k=cands_size)

        for docid in range(0, len(hits)):
            cands.append(int(hits[docid].docid))
            cands_score.append(hits[docid].score)
        # Append candidate answers
        tmp.append(cands)
        tmp.append(cands_score)
        dataset.append(tmp)

    return dataset

def get_dataset(query_path, labels_path, cands_size):
    """Splits the dataset into train, validation, and test set and creates
    the dataset form for training, validation, and testing.

    Returns:
        train_set: list of list in the form [qid, [pos ans], [ans candidates]]
        valid_set: list of list in the form [qid, [pos ans], [ans candidates]]
        test_set: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        query_path: str - path containing a list of qid and questions
        labels_path: str - path containing a list of qid and relevant docid
        cands_size: int - number of candidates to retrieve
    """
    # Question id and Question text
    queries = load_questions_to_df(query_path)
    # Question id and Answer id pair
    qid_docid = load_qid_docid_to_df(labels_path)
    # qid to docid label map
    labels = label_to_dict(qid_docid)
    train_label, test_label, valid_label = split_label(qid_docid)
    # Split Questions
    train_questions, test_questions, \
    valid_questions = split_question(train_label, test_label, valid_label, queries)

    print("\nGenerating training set...\n")
    train_set = create_dataset(train_questions, labels, cands_size)
    print("Generating validation set...\n")
    valid_set = create_dataset(valid_questions, labels, cands_size)
    print("Generating test set...\n")
    test_set = create_dataset(test_questions, labels, cands_size)

    return train_set, valid_set, test_set

def main():

    parser = argparse.ArgumentParser()
    # Required parameters
    parser.add_argument("--query_path", default=None, type=str, required=True,
    help="Path to the question id to text data in .tsv format. Each line should have at least two columns named (qid, question) separated by tab")
    parser.add_argument("--label_path", default=None, type=str, required=True,
    help="Path to the question id and answer id data in .tsv format. Each line should have at two columns named (qid, docid) separated by tab")

    # Optional parameters
    parser.add_argument("--cands_size", default=50, type=int, required=False,
    help="Number of candidates to retrieve per question.")
    parser.add_argument("--output_dir", default=Path.cwd()/'data/data_pickle/',
    type=str, required=False, help="The output directory where the generated data will be stored.")

    args = parser.parse_args()

    if len(sys.argv) < 4:
        print("Usage: python3 src/generate_data.py <query_path> <label_path>")
        sys.exit()

    train_set, valid_set, test_set = get_dataset(args.query_path, \
                                                 args.label_path, \
                                                 args.cands_size)

    save_pickle(args.output_dir + "train_set.pickle", train_set)
    save_pickle(args.output_dir + "valid_set.pickle", valid_set)
    save_pickle(args.output_dir + "test_set.pickle", test_set)

In [ ]:
#Define train dataset, validation dataset and test dataset
query_path = "/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/raw/FiQA_train_question_final.tsv"
labels_path = "/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/raw/FiQA_train_question_doc_final.tsv"
train_set, valid_set, test_set = get_dataset(query_path, labels_path, 50)


Generating training set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating validation set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating test set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
# Cleaning data
empty_docs, empty_id = get_empty_docs(collection)
# Remove empty answers from collection of answers
collection_cleaned = collection.drop(empty_id)
# Remove empty answers from qa pairs
qid_docid = qid_docid[~qid_docid['docid'].isin(empty_docs)]

print("Number of answers after cleaning: {}".format(len(collection_cleaned)))
print("Number of QA pairs after cleaning: {}".format(len(qid_docid)))

Number of answers after cleaning: 57600
Number of QA pairs after cleaning: 17072


In [ ]:
# Write collection df to file
save_tsv("retriever/collection_cleaned.tsv", collection_cleaned)

# Convert collection df to JSON file for Anserini's document indexer
collection_to_json("retriever/collection_json/docs.json", "retriever/collection_cleaned.tsv")

Metrics calculation

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score

def run_baseline(data, top_k):
  ap = []
  rr = []
  cg = []


  #for i, seq in enumerate(tqdm(train_set)):
  for i, seq in enumerate(data):
    qid, ans_labels, cands, cands_score = seq[0], seq[1], seq[2], seq[3]
    
    # Map question id to text
    #q_text = qid_to_text[qid]

    max_width = -1
    rr_ = 0
    ap_ = 0.0
    precision_ = 0.0
    relcnt_ = 0

    # top_k = 2

    # For each answer in the candidates
    for i in range(top_k):
      docid = cands[i]
      if docid in ans_labels and rr_ == 0:
        rr_ = 1/(i+1)


    relscores = [1 if docid in ans_labels else 0 for docid in cands[:top_k]]
    pos = [1.0/(i+1) for i in range(top_k)]
    ap_ = average_precision_score(relscores,pos) if sum(relscores) != 0 else 0 

    relscores = np.asarray([[cands_score[i] if (cands[i] in ans_labels) else 0.001 for i in range(top_k)]])
    pos = np.asarray([[np.log2(i+2) for i in range(top_k)]])


    # Our calculation of NDCG
    relscores_sorted = np.asarray([sorted(relscores[0], reverse=True)])
    dcg = 0
    idcg = 0
    for i in range(top_k):
      dcg += relscores[0][i]/pos[0][i]
      idcg += relscores_sorted[0][i]/pos[0][i]
    cg_ = dcg/idcg

    # cg_ = ndcg_score(relscores, pos)

    ap.append(ap_)
    rr.append(rr_)
    cg.append(cg_)
  return rr, ap, cg


      

In [ ]:
rr, ap, cg = run_baseline(train_set, top_k)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Mean Reciprocal Rank (MRR): 0.27206874303478207
Mean average Precision (MAP) 0.2701331456390404
Normalized Discounted Cumulative Gain (NDCG) 0.9516887851317029


Processing the questions and answers

In [ ]:
processed_answers = process_answers(collection_cleaned)
processed_questions = process_questions(queries)

In [ ]:
word2index, word2count = create_vocab(processed_answers, processed_questions)

print("Vocab size: {}".format(len(word2index)))
print("Top {} common words: {}".format(35, Counter(word2count).most_common(35)))

Vocab size: 85034
Top 35 common words: [('the', 371203), ('to', 233559), ('a', 201620), ('you', 166702), ('and', 163066), ('of', 157574), ('is', 129894), ('in', 120019), ('that', 111416), ('for', 89366), ('it', 83822), ('i', 74100), ('your', 68153), ('are', 67255), ('if', 60689), ('be', 59266), ('on', 58382), ('have', 55754), ('as', 50088), ('this', 49868), ('not', 49227), ('or', 46080), ('with', 45894), ('they', 44485), ('but', 41690), ('can', 38863), ('will', 36865), ('at', 35548), ('an', 31392), ('money', 31003), ('so', 29980), ('$', 29096), ('would', 28750), ('from', 28582), ('more', 27378)]


In [ ]:
qid_to_text, docid_to_text = id_to_text(collection, queries)
qid_to_tokenized_text, docid_to_tokenized_text = id_to_tokenized_text(processed_answers, processed_questions)

In [ ]:
pd.set_option('display.max_colwidth', None)
processed_questions.head()

,qid,question,q_processed,tokenized_q,q_len
0,0,What is considered a business expense on a business trip?,what is considered a business expense on a business trip,"[what, is, considered, a, business, expense, on, a, business, trip]",10
1,1,Claiming business expenses for a business with no income,claiming business expenses for a business with no income,"[claiming, business, expenses, for, a, business, with, no, income]",9
2,2,Transferring money from One business checking to another business checking,transferring money from one business checking to another business checking,"[transferring, money, from, one, business, checking, to, another, business, checking]",10
3,3,"Having a separate bank account for business/investing, but not a “business account?”",having a separate bank account for business investing but not a business account,"[having, a, separate, bank, account, for, business, investing, but, not, a, business, account]",13
4,4,Business Expense - Car Insurance Deductible For Accident That Occurred During a Business Trip,business expense car insurance deductible for accident that occurred during a business trip,"[business, expense, car, insurance, deductible, for, accident, that, occurred, during, a, business, trip]",13


In [ ]:
processed_answers.head()

,docid,doc,doc_processed,tokenized_ans,ans_len
0,3,"I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.",im not saying i dont like the idea of on the job training too but you cant expect the company to do that training workers is not their job theyre building software perhaps educational systems in the us or their students should worry a little about getting marketable skills in exchange for their massive investment in education rather than getting out with thousands in student debt and then complaining that they arent qualified to do anything,"[im, not, saying, i, dont, like, the, idea, of, on, the, job, training, too, but, you, cant, expect, the, company, to, do, that, training, workers, is, not, their, job, theyre, building, software, perhaps, educational, systems, in, the, us, or, their, students, should, worry, a, little, about, getting, marketable, skills, in, exchange, for, their, massive, investment, in, education, rather, than, getting, out, with, thousands, in, student, debt, and, then, complaining, that, they, arent, qualified, to, do, anything]",76
1,31,"So nothing preventing false ratings besides additional scrutiny from the market/investors, but there are some newer controls in place to prevent institutions from using them. Under the DFA banks can no longer solely rely on credit ratings as due diligence to buy a financial instrument, so that's a plus. The intent being that if financial institutions do their own leg work then *maybe* they'll figure out that a certain CDO is garbage or not. Edit: lead in",so nothing preventing false ratings besides additional scrutiny from the market investors but there are some newer controls in place to prevent institutions from using them under the dfa banks can no longer solely rely on credit ratings as due diligence to buy a financial instrument so thats a plus the intent being that if financial institutions do their own leg work then maybe theyll figure out that a certain cdo is garbage or not edit lead in,"[so, nothing, preventing, false, ratings, besides, additional, scrutiny, from, the, market, investors, but, there, are, some, newer, controls, in, place, to, prevent, institutions, from, using, them, under, the, dfa, banks, can, no, longer, solely, rely, on, credit, ratings, as, due, diligence, to, buy, a, financial, instrument, so, thats, a, plus, the, intent, being, that, if, financial, institutions, do, their, own, leg, work, then, maybe, theyll, figure, out, that, a, certain, cdo, is, garbage, or, not, edit, lead, in]",78
2,56,"You can never use a health FSA for individual health insurance premiums. Moreover, FSA plan sponsors can limit what they are will to reimburse. While you can't use a health FSA for premiums, you could previously use a 125 cafeteria plan to pay premiums, but it had to be a separate election from the health FSA. However, under N. 2013-54, even using a cafeteria plan to pay for indivdiual premiums is effectively prohibited.",you can never use a health fsa for individual health insurance premiums moreover fsa plan sponsors can limit what they are will to reimburse while you cant use a health fsa for premiums you could previously use a 125 cafeteria plan to pay premiums but it had to be a separate election from the health fsa however under n 2013 54 even using a cafeteria plan to pay for indivdiual premiums is effectively prohibited,"[you, can, never, use, a, health, fsa, for, individual, health, insurance, premiums, moreover, fsa, plan, sponsors, can, limit, what, they, are, will, to, reimburse, while, you, cant, use, a, health, fsa, for, premiums,

Training a model using BERT

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased') #generate tokens from BERT pretrained model

In [ ]:
from tqdm import tqdm


#bert_model = TFBertModel.from_pretrained('bert-base-cased')

def generate_bert_tokens(data, max_length, labels_count):
  input_ids = []
  token_type_ids = []
  att_masks = []
  labels = []
  data_modified = []


  #for i, seq in enumerate(tqdm(train_set)):
  for i, seq in enumerate(data):
    qid, ans_labels, cands = seq[0], seq[1], seq[2]
    
    # Map question id to text
    q_text = qid_to_text[qid]

    max_width = -1
   
    ans_labels_dict = {k: v for v, k in enumerate(ans_labels)}
    cands_valid = []
    cands_invalid = []
 
    for x in cands:
      if x in ans_labels_dict: # and pos_counter<pos_labels_count
        cands_valid.append(x)
        # pos_counter += 1
      else:   #  and neg_counter<neg_labels_count
        cands_invalid.append(x)
        # neg_counter += 1
    # cands_modified = cands_valid[:pos_labels_count] + cands_invalid[-neg_labels_count:]
    if len(cands_valid) >= labels_count:
      cands_modified = cands_valid[:labels_count]
    else:
      cands_modified = cands_valid + cands_invalid[-(labels_count-len(cands_valid)):]
 
    # For each answer in the chosen candidates
    for docid in cands_modified:
      # Map the docid to text
      ans_text = docid_to_text[docid] if docid_to_text.get(docid) else ''

      # Encode the sequence using BERT tokenizer
      encoded_seq = bert_tokenizer.encode_plus(q_text, ans_text[:500],
                                              max_length=max_length,
                                              pad_to_max_length=True,
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_attention_mask = True)

 

      # Get parameters
      input_ids.append(encoded_seq['input_ids'])
      token_type_ids.append(encoded_seq['token_type_ids'])
      att_masks.append(encoded_seq['attention_mask'])

      # If an answer is in the list of relevant answers assign
      # positive label
      labels.append(1 if docid in ans_labels else 0)
      data_modified.append([qid, ans_labels, docid])


      #max_width = max(max_width, len(ans_text+q_text))
      #print(max_width)
  return input_ids, token_type_ids, att_masks, labels, data_modified

Generate BERT tokens by dividing the training dataset into several buckets. This was done to improve performance.First we save the output from BERT tokenizer as pickle files and later we load them as a list and concatenate them.

In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(train_set[:1000], max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids1.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids1.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks1.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels1.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified1.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(train_set[1000:2000], max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids2.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids2.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks2.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels2.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified2.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(train_set[2000:3000], max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids3.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids3.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks3.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels3.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified3.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(train_set[3000:4000], max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids4.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids4.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks4.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels4.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified4.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(train_set[4000:], max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids5.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids5.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks5.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels5.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified5.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:
# save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids5.pickle", input_ids)
# save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids5.pickle", token_type_ids)
# save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks5.pickle", att_masks)

In [ ]:
id1 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids1.pickle")
type1 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids1.pickle")
mask1 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks1.pickle")
label1 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels1.pickle")
data_modified1 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified1.pickle")
print(len(id1), len(type1), len(mask1), len(label1), len(data_modified1))
print(sum(max(mask1, key = sum)))

4000 4000 4000 4000 4000
228


In [ ]:
# labels0 = []
# (bad, good)[x in goodvals].append(x)
# print(len(label1), sum(label1), label1[:100])
print(len(id1[0]), id1[0])
print(len(type1[0]), type1[0])
print(len(mask1[0]), mask1[0])
print(sum(max(mask1, key = sum)))

256 [101, 1327, 1110, 1737, 170, 1671, 11013, 1113, 170, 1671, 3868, 136, 102, 1109, 146, 8900, 144, 16423, 3923, 22383, 1106, 1103, 2548, 119, 1130, 1704, 1103, 1436, 146, 1169, 1474, 1110, 1240, 1671, 11013, 1336, 1129, 1260, 13890, 5225, 119, 1252, 1122, 9113, 1113, 1103, 5607, 1105, 1184, 1122, 1110, 1128, 1328, 1106, 1260, 13890, 119, 7938, 13429, 27452, 1150, 3201, 1283, 1121, 1313, 1113, 1671, 1336, 1260, 13890, 2272, 11928, 117, 1259, 1103, 2616, 1104, 3634, 1147, 7680, 117, 1103, 2616, 1104, 25338, 13556, 1105, 13077, 1105, 1168, 6655, 1105, 3238, 11928, 119, 13429, 27452, 1132, 1737, 789, 6934, 1283, 1121, 1313, 790, 1191, 1147, 5078, 4752, 1172, 1106, 1129, 1283, 175, 1197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
id2 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids2.pickle")
type2 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids2.pickle")
mask2 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks2.pickle")
label2 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels2.pickle")
data_modified2 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified2.pickle")
print(len(id2), len(type2), len(mask2), len(label2))
print(sum(max(mask2, key = sum)))

4000 4000 4000 4000
227


In [ ]:
id3 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids3.pickle")
type3 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids3.pickle")
mask3 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks3.pickle")
label3 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels3.pickle")
data_modified3 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified3.pickle")
print(len(id3), len(type3), len(mask3), len(label3))
print(sum(max(mask3, key = sum)))

4000 4000 4000 4000
256


In [ ]:
id4 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids4.pickle")
type4 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids4.pickle")
mask4 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks4.pickle")
label4 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels4.pickle")
data_modified4 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified4.pickle")
print(len(id4), len(type4), len(mask4), len(label4))
print(sum(max(mask4, key = sum)))

4000 4000 4000 4000
229


In [ ]:
id5 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids5.pickle")
type5 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_ids5.pickle")
mask5 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks5.pickle")
label5 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels5.pickle")
data_modified5 = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified5.pickle")
print(len(id5), len(type5), len(mask5), len(label5))
print(sum(max(mask5, key = sum)))

6732 6732 6732 6732
256


In [ ]:
id_all = id1 + id2 + id3 + id4 + id5
type_all = type1 + type2 + type3 + type4 + type5
mask_all = mask1 + mask2 + mask3 + mask4 + mask5
label_all = label1 + label2 + label3 + label4 + label5
data_modified_all = data_modified1 + data_modified2 + data_modified3 + data_modified4 + data_modified5
print(len(id_all), len(type_all), len(mask_all), len(label_all))
print(sum(max(mask_all, key = sum)))

22732 22732 22732 22732
256


In [ ]:
input_ids, token_type_ids, att_masks, labels, data_modified = generate_bert_tokens(valid_set, max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids_val.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_val.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks_val.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels_val.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified_val.pickle", data_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
id_val = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids_val.pickle")
type_val = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_val.pickle")
mask_val = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks_val.pickle")
label_val = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels_val.pickle")
data_modified_val = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "data_modified_val.pickle")
print(len(id_val), len(type_val), len(mask_val), len(label_val))
print(sum(max(mask_val, key = sum)))

2528 2528 2528 2528
237


In [ ]:
input_ids, token_type_ids, att_masks, labels, test_set_modified = generate_bert_tokens(test_set, max_length, labels_count)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids_test.pickle", input_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_test.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks_test.pickle", att_masks)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels_test.pickle", labels)
save_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "test_set_modified.pickle", test_set_modified)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
id_test = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "input_ids_test.pickle")
type_test = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "token_type_test.pickle")
mask_test = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "att_masks_test.pickle")
label_test = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "labels_test.pickle")
test_set_modified = load_pickle('/gdrive/MyDrive/w266-final-project-master/w266-final-project-master/FinBERT-QA/data/data_pickle/' + "test_set_modified.pickle")

print(len(id_test), len(type_test), len(mask_test), len(label_test), len(test_set_modified))
print(sum(max(mask_test, key = sum)))

999 999 999 999 999
226


Creating a BERT classification model 

In [ ]:
def create_bert_model(max_length):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """
    
    ### YOUR CODE HERE
    train_layers = 2
    hidden_size = 100
    dropout = 0.3
    learning_rate= 0.00005
    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    # max_length = 64

    # #restrict training to the train_layers outer transformer layers
    # if not train_layers == -1:

    #         retrain_layers = []

    #         for retrain_layer_number in range(train_layers):

    #             layer_code = '_' + str(11 - retrain_layer_number)
    #             retrain_layers.append(layer_code)

    #         for w in bert_model.weights:
    #             if not any([x in w.name for x in retrain_layers]):
    #                 w._trainable = False

    for i in range(12-train_layers):
      bert_model.bert.encoder.layer[i].trainable = False

    #restrict training to the train_layers outer transformer layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer') #--SOLUTION--
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}         

    bert_out = bert_model(bert_inputs) 

    cls_token = bert_out[0][:,0,:]
    #cls_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                            metrics='accuracy') 
    ### END YOUR CODE
    
    return classification_model

In [ ]:
model = create_bert_model(max_length) 

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
#clean up
del input_ids
del token_type_ids
del att_masks
del labels

Checking the the size of the embeddings

In [ ]:
n = len(id1[0])
for i in range(len(id1)):
  if len(id1[i]) != n:
    print(i, len(id1[i]))
    break
print(n)

n = len(type1[0])
for i in range(len(type1)):
  if len(type1[i]) != n:
    print(i, len(type1[i]))
    break
print(n)

n = len(mask1[0])
for i in range(len(mask1)):
  if len(mask1[i]) != n:
    print(i, len(mask1[i]))
    break
print(n)

n = len(label1)


256
256
256


In [ ]:
# bert_avg_model_history = model.fit([np.array(id1[:100]), np.array(type1[:100]), np.array(mask1[:100])],np.array(label1[:100]),
#                                    validation_data=([np.array(id1[:10]), np.array(type1[:10]), np.array(mask1[:10])],np.array(label1[:10])),
#                                    batch_size=8,epochs=1)  

Model fitting and saving weights

In [ ]:
bert_avg_model_all = model.fit([np.array(id_all), np.array(type_all), np.array(mask_all)],np.array(label_all),
                                   validation_data=([np.array(id_val), np.array(type_val), np.array(mask_val)],np.array(label_val)),
                                   batch_size=8,epochs=1) 
model.save_weights(f"bert_avg_model_L{labels_count}_{max_length}.h5")
print(model.summary())

Creating BERT classification model and loading weights from saved model

In [ ]:
model2 = create_bert_model(max_length)
# model2.load_weights(f"bert_avg_model_L{labels_count}_{max_length}.h5")
model2.load_weights(f"model_weight_{labels_count}_{max_length}.h5")
print(model2.summary())

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 256)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 256)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 256)]       0           []                               
 er)                                                                                              
                                                                                            

Model prediction with test data

In [ ]:
# bert_avg_model_predict = model.predict([np.array(id_test), np.array(type_test), np.array(mask_test)]) 
bert_avg_model_predict = model2.predict([np.array(id_test), np.array(type_test), np.array(mask_test)]) 

In [ ]:
# bert_avg_model_predict[0]


Test Model accuracy

In [ ]:
print(len(bert_avg_model_predict))
print(np.squeeze(bert_avg_model_predict[:10]))
print(label_test[:10])
print(id_test[:10])
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(label_test, np.squeeze(bert_avg_model_predict))
print('Test accuracy:', float(m.result()))


999
[0.02918151 0.03474799 0.1118913  0.27082923 0.07827142 0.09202135
 0.01853544 0.07087332 0.6603873  0.00728986]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[[101, 9743, 1139, 2252, 112, 188, 1671, 2653, 1139, 1671, 136, 102, 2066, 1121, 1139, 1319, 2541, 113, 146, 1821, 1136, 1126, 23195, 114, 131, 1130, 1901, 1106, 11128, 1112, 112, 1671, 2467, 112, 113, 9377, 1568, 1413, 1367, 164, 122, 166, 114, 1240, 109, 11975, 113, 1137, 3451, 114, 1209, 1129, 2548, 1106, 199, 1405, 110, 2191, 118, 6233, 3641, 117, 1113, 23070, 12342, 119, 1188, 7450, 1106, 1240, 9377, 1568, 1413, 199, 4667, 117, 1134, 1110, 1170, 1155, 1240, 112, 27939, 1106, 2467, 112, 117, 22346, 1116, 117, 1105, 1260, 11243, 1116, 118, 1177, 117, 1343, 1274, 112, 189, 4851, 1122, 119, 8636, 1104, 1103, 1405, 110, 1110, 1260, 13890, 5225, 1113, 1413, 199, 1765, 117, 1191, 1128, 1138, 1536, 27522, 2165, 2467, 1111, 1122, 1106, 2187, 132, 1133, 117, 1107, 1251, 1692, 117, 1128, 1209, 12972, 1120, 1655, 122, 120, 123, 1104, 1103, 1405, 1

Model evaluation with Ranking Metrics

In [ ]:
print(len(test_set_modified), len(id_test))

temp = pd.DataFrame(test_set_modified)
temp.columns = ['qid', 'ans_label', 'cand']
temp['bert_score'] = np.squeeze(bert_avg_model_predict)
temp2 = temp.groupby('qid').agg({'ans_label': 'first', 'cand': lambda x: tuple(x), 'bert_score': lambda x: tuple(x)}).applymap(list).reset_index()
print(temp.head(20))
test_set_bert_score_list = temp2.values.tolist()
print(test_set_bert_score_list[:3])

rr, ap, cg = run_baseline(test_set_bert_score_list, top_k)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))



999 999
    qid                         ans_label    cand  bert_score
0    48  [108062, 401260, 329810, 512151]  512151    0.029182
1    48  [108062, 401260, 329810, 512151]  252843    0.034748
2    48  [108062, 401260, 329810, 512151]  272279    0.111891
3    54          [590775, 109546, 511651]  590775    0.270829
4    54          [590775, 109546, 511651]  192843    0.078271
5    54          [590775, 109546, 511651]  272279    0.092021
6    60                          [381151]  291717    0.018535
7    60                          [381151]  524134    0.070873
8    60                          [381151]  183500    0.660387
9    80                          [252473]  210673    0.007290
10   80                          [252473]  170511    0.016846
11   80                          [252473]   37070    0.008381
12  444  [573518, 175824, 483282, 479203]  175824    0.154781
13  444  [573518, 175824, 483282, 479203]  479203    0.861560
14  444  [573518, 175824, 483282, 479203]  573518    0.926755


Example Question, Answer from the above result (ranking)

In [ ]:
print(qid_to_text[444])
print(docid_to_text[573518],'\n', docid_to_text[479203],'\n', docid_to_text[175824])

Why do most banks in Canada charge monthly fee?
Arguably, "because they can".  Canada's banking industry is dominated by five chartered banks who by virtue of their size, pretty much determine how banking is done in Canada. Yes, they have to abide by government regulation, but they carry enough weight to influence government and to some extent shape the regulation they have to follow. While this situation makes Canada's financial system very stable and efficient, it also permits anti-competitive behavior.  There was a time (when U.S. banks were not permitted to operate across state lines) when the smallest of Canada's "big 5" was bigger than the biggest U.S. bank, despite our economy having always been about 1/10 the size of the U.S. That scale and their small number gives the "big 5" the ability to invest heavily in and collaborate on whatever they decide to be in their own interest. So, if they want to charge fees, they do. 
 You have to check your contract to be sure what is it you'